### This notebook includes non-rigid tranformations using Affine

In [1]:
# Import Libraries
import os
from pathlib import Path
import numpy as np
import pandas as pd
import SimpleITK as sitk
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
from utils_c import *
import re

In [2]:
# get the current working directory
cwd = os.getcwd()
print(cwd)
# go over the data directory
data_dir = Path(cwd).parent / 'data' / 'nifti' 
print(data_dir)

d:\VS_Projects\MIRA_FINAL_PROJECT\notebooks
d:\VS_Projects\MIRA_FINAL_PROJECT\data\nifti


In [3]:
# Create affine and b-spline directories
if not os.path.exists(f"../data/affine"):
    os.makedirs(f"../data/affine")

# Create directories for the results
for k in range(1,5):
    if not os.path.exists(f"../data/affine/reg_results{k}"):
        os.makedirs(f"../data/affine/reg_results{k}")
    if not os.path.exists(f"../data/bspline/reg_results{k}"):
        os.makedirs(f"../data/bspline/reg_results{k}")

In [4]:
# Get the paths for the images and landmarks
def get_paths(n):
    # Set inhale image as fixed image and exhale image as moving image
    img_f, landmark_f = get_data_paths(p="i", i=n)
    img_m, landmark_m = get_data_paths(p="e", i=n)

    return img_f, img_m, landmark_f, landmark_m

## Affine Transformation

In [5]:
# apply affine transformation
img_f, img_m, _, _ = get_paths(1)
!elastix -f {img_f} -m {img_m} -p ../data/param_affine.txt -out ../data/affine/reg_results1


elastix is started at Tue Dec  6 22:36:07 2022.

which elastix:   elastix
elastix runs at: DESKTOP-UP0FCMD
  Windows  Professional (x64),  (Build 9200)
  with 16258 MB memory, and 2 cores @ 2905 MHz.
-------------------------------------------------------------------------

Running elastix with parameter file 0: "../data/param_affine.txt".

Current time: Tue Dec  6 22:36:07 2022.
Reading the elastix parameters from file ...

Installing all components.
InstallingComponents was successful.

ELASTIX version: 5.000
Command line options from ElastixBase:
-f        ../data/nifti/copd1_iBHCT.nii.gz
-m        ../data/nifti/copd1_eBHCT.nii.gz
-fMask    unspecified, so no fixed mask used
-mMask    unspecified, so no moving mask used
-out      ..\data\affine\reg_results1\
-p        ../data/param_affine.txt
-priority unspecified, so NORMAL process priority
-threads  unspecified, so all available threads are used
Command line options from TransformBase:
-t0       unspecified, so no initial transfo

In [6]:
img_f, img_m, _, _ = get_paths(2)
!elastix -f {img_f} -m {img_m} -p ../data/param_affine.txt -out ../data/affine/reg_results2


elastix is started at Tue Dec  6 22:37:15 2022.

which elastix:   elastix
elastix runs at: DESKTOP-UP0FCMD
  Windows  Professional (x64),  (Build 9200)
  with 16258 MB memory, and 2 cores @ 2904 MHz.
-------------------------------------------------------------------------

Running elastix with parameter file 0: "../data/param_affine.txt".

Current time: Tue Dec  6 22:37:15 2022.
Reading the elastix parameters from file ...

Installing all components.
InstallingComponents was successful.

ELASTIX version: 5.000
Command line options from ElastixBase:
-f        ../data/nifti/copd2_iBHCT.nii.gz
-m        ../data/nifti/copd2_eBHCT.nii.gz
-fMask    unspecified, so no fixed mask used
-mMask    unspecified, so no moving mask used
-out      ..\data\affine\reg_results2\
-p        ../data/param_affine.txt
-priority unspecified, so NORMAL process priority
-threads  unspecified, so all available threads are used
Command line options from TransformBase:
-t0       unspecified, so no initial transfo

In [7]:
img_f, img_m, _, _ = get_paths(3)
!elastix -f {img_f} -m {img_m} -p ../data/param_affine.txt -out ../data/affine/reg_results3


elastix is started at Tue Dec  6 22:37:59 2022.

which elastix:   elastix
elastix runs at: DESKTOP-UP0FCMD
  Windows  Professional (x64),  (Build 9200)
  with 16258 MB memory, and 2 cores @ 2904 MHz.
-------------------------------------------------------------------------

Running elastix with parameter file 0: "../data/param_affine.txt".

Current time: Tue Dec  6 22:37:59 2022.
Reading the elastix parameters from file ...

Installing all components.
InstallingComponents was successful.

ELASTIX version: 5.000
Command line options from ElastixBase:
-f        ../data/nifti/copd3_iBHCT.nii.gz
-m        ../data/nifti/copd3_eBHCT.nii.gz
-fMask    unspecified, so no fixed mask used
-mMask    unspecified, so no moving mask used
-out      ..\data\affine\reg_results3\
-p        ../data/param_affine.txt
-priority unspecified, so NORMAL process priority
-threads  unspecified, so all available threads are used
Command line options from TransformBase:
-t0       unspecified, so no initial transfo

In [8]:
img_f, img_m, _, _ = get_paths(4)
!elastix -f {img_f} -m {img_m} -p ../data/param_affine.txt -out ../data/affine/reg_results4


elastix is started at Tue Dec  6 22:38:51 2022.

which elastix:   elastix
elastix runs at: DESKTOP-UP0FCMD
  Windows  Professional (x64),  (Build 9200)
  with 16258 MB memory, and 2 cores @ 2904 MHz.
-------------------------------------------------------------------------

Running elastix with parameter file 0: "../data/param_affine.txt".

Current time: Tue Dec  6 22:38:51 2022.
Reading the elastix parameters from file ...

Installing all components.
InstallingComponents was successful.

ELASTIX version: 5.000
Command line options from ElastixBase:
-f        ../data/nifti/copd4_iBHCT.nii.gz
-m        ../data/nifti/copd4_eBHCT.nii.gz
-fMask    unspecified, so no fixed mask used
-mMask    unspecified, so no moving mask used
-out      ..\data\affine\reg_results4\
-p        ../data/param_affine.txt
-priority unspecified, so NORMAL process priority
-threads  unspecified, so all available threads are used
Command line options from TransformBase:
-t0       unspecified, so no initial transfo

#### Transform Landmarks using Transformix from Elastix

In [9]:
# Create folders for affine transformation results 
for z in range(1,5):
    if not os.path.exists(f"../data/affine/reg_results{z}/registered_moving"):
        os.makedirs(f"../data/affine/reg_results{z}/registered_moving")

for m in range(1,5):
    if not os.path.exists(f"../data/affine/reg_results{m}/registered_landmarks"):
        os.makedirs(f"../data/affine/reg_results{m}/registered_landmarks")

In [10]:
# fixed image = inhale image and moving image = exhale image
img_f, img_m, landmark_f, landmark_m = get_paths(1)

!transformix -in {img_m} -tp ../data/affine/reg_results1/TransformParameters.0.txt -out ../data/affine/reg_results1/registered_moving
!transformix -def {landmark_f} -tp ../data/affine/reg_results1/TransformParameters.0.txt -out ../data/affine/reg_results1/registered_landmarks


transformix is started at Tue Dec  6 22:40:00 2022.

which transformix:   transformix
transformix runs at: DESKTOP-UP0FCMD
  Windows  Professional (x64),  (Build 9200)
  with 16258 MB memory, and 2 cores @ 2904 MHz.
Running transformix with parameter file "../data/affine/reg_results1/TransformParameters.0.txt".

Reading the elastix parameters from file ...

Installing all components.
InstallingComponents was successful.

ELASTIX version: 5.000
Command line options from ElastixBase:
-in       ../data/nifti/copd1_eBHCT.nii.gz
-out      ..\data\affine\reg_results1\registered_moving\
-threads  unspecified, so all available threads are used
-tp       ../data/affine/reg_results1/TransformParameters.0.txt
-def      unspecified, so no input points transformed
-jac      unspecified, so no det(dT/dx) computed
-jacmat   unspecified, so no dT/dx computed

Reading input image ...
  Reading input image took 0.590518 s
Calling all ReadFromFile()'s ...
  Calling all ReadFromFile()'s took 0.000423 s
T

In [11]:
# fixed image = inhale image and moving image = exhale image
img_f, img_m, landmark_f, landmark_m = get_paths(2)

!transformix -in {img_m} -tp ../data/affine/reg_results2/TransformParameters.0.txt -out ../data/affine/reg_results2/registered_moving
!transformix -def {landmark_f} -tp ../data/affine/reg_results2/TransformParameters.0.txt -out ../data/affine/reg_results2/registered_landmarks


transformix is started at Tue Dec  6 22:40:45 2022.

which transformix:   transformix
transformix runs at: DESKTOP-UP0FCMD
  Windows  Professional (x64),  (Build 9200)
  with 16258 MB memory, and 2 cores @ 2904 MHz.
Running transformix with parameter file "../data/affine/reg_results2/TransformParameters.0.txt".

Reading the elastix parameters from file ...

Installing all components.
InstallingComponents was successful.

ELASTIX version: 5.000
Command line options from ElastixBase:
-in       ../data/nifti/copd2_eBHCT.nii.gz
-out      ..\data\affine\reg_results2\registered_moving\
-threads  unspecified, so all available threads are used
-tp       ../data/affine/reg_results2/TransformParameters.0.txt
-def      unspecified, so no input points transformed
-jac      unspecified, so no det(dT/dx) computed
-jacmat   unspecified, so no dT/dx computed

Reading input image ...
  Reading input image took 0.446833 s
Calling all ReadFromFile()'s ...
  Calling all ReadFromFile()'s took 0.000160 s
T

In [12]:
# fixed image = inhale image and moving image = exhale image
img_f, img_m, landmark_f, landmark_m = get_paths(3)

!transformix -in {img_m} -tp ../data/affine/reg_results3/TransformParameters.0.txt -out ../data/affine/reg_results3/registered_moving
!transformix -def {landmark_f} -tp ../data/affine/reg_results3/TransformParameters.0.txt -out ../data/affine/reg_results3/registered_landmarks


transformix is started at Tue Dec  6 22:41:09 2022.

which transformix:   transformix
transformix runs at: DESKTOP-UP0FCMD
  Windows  Professional (x64),  (Build 9200)
  with 16258 MB memory, and 2 cores @ 2904 MHz.
Running transformix with parameter file "../data/affine/reg_results3/TransformParameters.0.txt".

Reading the elastix parameters from file ...

Installing all components.
InstallingComponents was successful.

ELASTIX version: 5.000
Command line options from ElastixBase:
-in       ../data/nifti/copd3_eBHCT.nii.gz
-out      ..\data\affine\reg_results3\registered_moving\
-threads  unspecified, so all available threads are used
-tp       ../data/affine/reg_results3/TransformParameters.0.txt
-def      unspecified, so no input points transformed
-jac      unspecified, so no det(dT/dx) computed
-jacmat   unspecified, so no dT/dx computed

Reading input image ...
  Reading input image took 0.609237 s
Calling all ReadFromFile()'s ...
  Calling all ReadFromFile()'s took 0.000172 s
T

In [13]:
# fixed image = inhale image and moving image = exhale image
img_f, img_m, landmark_f, landmark_m = get_paths(4)

!transformix -in {img_m} -tp ../data/affine/reg_results4/TransformParameters.0.txt -out ../data/affine/reg_results4/registered_moving
!transformix -def {landmark_f} -tp ../data/affine/reg_results4/TransformParameters.0.txt -out ../data/affine/reg_results4/registered_landmarks


transformix is started at Tue Dec  6 22:41:37 2022.

which transformix:   transformix
transformix runs at: DESKTOP-UP0FCMD
  Windows  Professional (x64),  (Build 9200)
  with 16258 MB memory, and 2 cores @ 5475 MHz.
Running transformix with parameter file "../data/affine/reg_results4/TransformParameters.0.txt".

Reading the elastix parameters from file ...

Installing all components.
InstallingComponents was successful.

ELASTIX version: 5.000
Command line options from ElastixBase:
-in       ../data/nifti/copd4_eBHCT.nii.gz
-out      ..\data\affine\reg_results4\registered_moving\
-threads  unspecified, so all available threads are used
-tp       ../data/affine/reg_results4/TransformParameters.0.txt
-def      unspecified, so no input points transformed
-jac      unspecified, so no det(dT/dx) computed
-jacmat   unspecified, so no dT/dx computed

Reading input image ...
  Reading input image took 1.544828 s
Calling all ReadFromFile()'s ...
  Calling all ReadFromFile()'s took 0.000366 s
T

In [15]:
## Extract the landmarks from the registered landmarks files
def extract_landmarks (filepath):

    new_landmarks = np.zeros((300, 3))
    current_landmark_file = open(filepath, "r")
    reg_ex = r'OutputIndexFixed = \[([\d.\s\-]+)\]'

    for i, line in enumerate(current_landmark_file):
        
        match_found = re.search(reg_ex, line, re.M)
        j = match_found.group(1).split()
        j = [round(float(c)) for c in j]
        new_landmarks[i,:] = j
        
    return new_landmarks
    
def calculate_tre(fixed_points, moving_points):
    tre = np.sqrt(np.sum((fixed_points - moving_points) ** 2, axis=1))
    return tre

def add_300(filepath):
    header = ["300"]
    with open(filepath,'r+') as infile:
        content = infile.read()
        infile.seek(0,0)
        infile.write("\t".join(header)+"\n"+content)

In [83]:
# Calculate TRE for all the patients

inhale_img = sitk.ReadImage("../data/nifti/copd1_iBHCT.nii.gz")
x,y,z = inhale_img.GetSpacing()

moving_points = np.loadtxt("../data/keypoints/copd1_300_eBH_xyz_r1.txt")
registered_inhale_points_to_exhale = extract_landmarks("../data/affine/reg_results1/registered_landmarks/outputpoints.txt")

moving_points = moving_points * np.array([x,y,z])
fixed_points = registered_inhale_points_to_exhale * np.array([x,y,z])

mean = np.mean(calculate_tre(fixed_points, moving_points))
std = np.std(calculate_tre(fixed_points, moving_points))
print(f"Mean: {mean}, Std: {std}")

Mean: 25.777313343905625, Std: 11.274261771305325


In [19]:
def calculate_tre_all(i):
    img_f, img_m, landmark_f, landmark_m = get_paths(i)
    inhale_img = sitk.ReadImage(img_f)
    x,y,z = inhale_img.GetSpacing()

    #add_300(landmark_f) --> instead manually add 300 in the inhale landmark files
    moving_points = np.loadtxt(landmark_m)
    registered_inhale_points = extract_landmarks(f"../data/affine/reg_results{i}/registered_landmarks/outputpoints.txt")
    
    moving_points = moving_points * np.array([x,y,z])
    fixed_points = registered_inhale_points* np.array([x,y,z])
    
    mean = np.mean(calculate_tre(fixed_points, moving_points))
    std = np.std(calculate_tre(fixed_points, moving_points))
    print(f"Patient {i}: Mean: {mean}, Std: {std}")


In [20]:
for p in range(1,5):
    calculate_tre_all(p)

Patient 1: Mean: 25.777313343905625, Std: 11.274261771305325
Patient 2: Mean: 22.09070810780162, Std: 6.103527586920725
Patient 3: Mean: 8.213236600401755, Std: 3.3003394037191316
Patient 4: Mean: 26.63573292258854, Std: 10.557805925145093
